**Import required libraries and scripts**

In [ ]:
#Import required libraries and scripts
from scripts.library_preparation import *
from scripts.utilities import *
from scripts.docking_functions import *
from scripts.clustering_functions import *
from scripts.rescoring_functions import *
from scripts.consensus_methods import *
from scripts.performance_calculation import *
from scripts.dogsitescorer import *
from scripts.get_pocket import *
from scripts.postprocessing import *

In [ ]:
software = Path('/home/tony/DockM8/software')
receptors = [Path('/home/tony/DockM8/testing_ensemble_docking/protein1.pdb'), Path('/home/tony/DockM8/testing_ensemble_docking/protein2.pdb')]
ref_files = [Path('/home/tony/DockM8/testing_ensemble_docking/ref1.sdf'), Path('/home/tony/DockM8/testing_ensemble_docking/ref2.sdf')]
docking_library = Path('/home/tony/DockM8/testing_ensemble_docking/library.sdf')
docking_programs = ['PLANTS']
clustering_metrics = ['bestpose_PLANTS']
rescoring_functions = ['GNINA_Affinity','CNN-Score','CNN-Affinity', 'AD4', 'CHEMPLP', 'RFScoreVS']
id_column = 'ID'
n_poses = 10
exhaustiveness = 8
protonation = 'GypsumDL'
ncpus = 5
pocket = 'reference'


print('DockM8 is running in ensemble mode...')
        
receptor_dict = {}
for i, receptor in enumerate(receptors):
    receptor_dict[receptor] = ref_files[i]
    
for receptor, ref_file in receptor_dict.items():

    w_dir = Path(receptor).parent / Path(receptor).stem
    print('The working directory has been set to:', w_dir)
    (w_dir).mkdir(exist_ok=True)
    
    #if os.path.isfile(str(receptor).replace('.pdb', '_pocket.pdb')) == False:
    if pocket == 'reference':
        pocket_definition = get_pocket(ref_file, receptor, 8)
        print(pocket_definition)
    if pocket == 'RoG':
        pocket_definition = get_pocket_RoG(ref_file, receptor)
        print(pocket_definition)
    elif pocket == 'dogsitescorer':
        pocket_definition = binding_site_coordinates_dogsitescorer(receptor, w_dir, method='volume')
        print(pocket_definition)
    #else:
        #pocket_definition = calculate_pocket_coordinates_from_pocket_pdb_file((str(receptor).replace('.pdb', '_pocket.pdb')))
        
    if (w_dir / 'final_library.sdf').is_file() == False:
        prepare_library(docking_library, w_dir, id_column, protonation, software, ncpus)
        
    docking(w_dir, receptor, pocket_definition, software, docking_programs, exhaustiveness, n_poses, 5)
    concat_all_poses(w_dir, docking_programs)

    print('Loading all poses SDF file...')
    tic = time.perf_counter()
    all_poses = PandasTools.LoadSDF(str(w_dir / 'allposes.sdf'), idName='Pose ID', molColName='Molecule', includeFingerprints=False, strictParsing=True)
    print(f'Loaded {len(all_poses)} poses.')
    toc = time.perf_counter()
    print(f'Finished loading all poses SDF in {toc-tic:0.4f}!...')

    for metric in clustering_metrics:
            if os.path.isfile(w_dir / 'clustering/'f'{metric}_clustered.sdf') == False:
                cluster_pebble(metric, 'KMedoids', w_dir, receptor, all_poses, ncpus)
                
    for metric in clustering_metrics:
            rescore_all(w_dir, receptor, pocket_definition, software, w_dir / 'clustering' / f'{metric}_clustered.sdf', rescoring_functions , ncpus)

    apply_consensus_methods(w_dir, 'bestpose_PLANTS', 'method6', ['GNINA_Affinity','CNN-Score','CNN-Affinity', 'AD4', 'CHEMPLP', 'RFScoreVS'], 'min_max')